In [3]:
import networkx as nx
import sys
import time
import pandas as pd
import numpy as np
from scipy.special import loggamma
import matplotlib.pyplot as plt
import gym

In [4]:
# scale_gradient = lambda d, L2_max: min(L2_max / np.linalg.norm(d), 1) * d
# class GradientQLearning:
#     def __init__(self, n_actions, discount, lr):
#         self.n_actions = n_actions
#         self.action_space = [0 for _ in range(n_actions)]
#         self.discount_rate = discount # gamma
#         self.Q = Q # action value function Q(theta, s, a)
#         self.dQ = None
#         self.theta = None # action value function parameter
#         self.lr = lr # alpha: learning rate

# def lookahead(model, s, a):
#     return model.Q(model.theta, s, a)

# def update(m, s, a, r, sp):
#     u = max(m.Q(m.theta, sp, ap) for ap in m.action_space)
#     d = (r + m.discount_rate*u - m.Q(m.theta, s, a)) * m.dQ(m.theta, s, a)
#     m.theta += m.lr * scale_gradient(d, 1)
#     return m

In [5]:
# class MDP:
#     def __init__(self, n_states, n_actions, discount, lr):
#         self.n_states = n_states
#         self.n_actions = n_actions
#         self.state_space = np.zeros((n_states)) # [0 for _ in range(n_states)]
#         self.action_space = np.zeros((n_actions)) # [0 for _ in range(n_actions)]
#         self.discount_rate = discount # gamma
#         self.reward_func = None
#         self.transition_reward = None
 

# def simulate(mdp, model, policy, horizon, s):
#     for _ in range(horizon):
#         a = policy(model, s)
#         sp, r = mdp.tr(s,a)
#         update(model, s, a, r, sp)
#         s = sp

# class PolicyIteration:
#     def __init__(self, k_max, n_states, n_actions):
#         self.policy = np.random.randint(0, n_actions, n_states) # initialize random actions in each state
#         self.k_max = k_max
    
# def policy_evaluation(mdp, policy):
#     rewardsp = [mdp.reward_func(s, policy[s]) for s in mdp.state_space]
#     transitionp = [mdp]
    
# def solve(model, mdp):
#     for _ in range(model.k_max):
#         utility = policy_evaluation(mdp, model.policy)
#         policyp = ValueFunctionPolicy(mdp, utility)
#         if np.all(model.policy[s] == policyp[s] for s in mdp.state_space):
#             break
#         model.policy = policyp
#     return model.policy

In [54]:

class QLearning:
    def __init__(self, n_states, n_actions, discount, lr, initial_Q_value):
        self.n_states = n_states
        self.n_actions = n_actions
        self.state_space = np.zeros((n_states)) # [0 for _ in range(n_states)]
        self.action_space = np.zeros((n_actions)) # [0 for _ in range(n_actions)]
        self.discount_rate = discount # gamma
        self.Q = np.zeros((n_states, n_actions)) # action value function Q(s, a)
        self.Q.fill(initial_Q_value) 
        self.lr = lr # alpha: learning rate

lookahead = lambda m, s, a: m.Q[s, a]
def update(m, s, a, r, sp):
    if m.Q[s,a] == -np.inf:
        m.Q[s,a] = 0.
    m.Q[s, a] += m.lr * (r + m.discount_rate * np.max(m.Q[sp]) - m.Q[s,a])
    return m

In [64]:
dataset = 'medium'
df = pd.read_csv(f'data/{dataset}.csv')
print(df.nunique()['s'], df['s'].max())
df['s'] -= 1
df['a'] -= 1
df['sp'] -= 1
# counts = df.nunique()
if dataset == 'medium':
    model = QLearning(50000, df['a'].max()+1, 1., 0.01, -np.inf)
else:
    model = QLearning(df['s'].max()+1, df['a'].max()+1, 0.95, 0.01, -np.inf)
print(model.Q.shape)
for i, obs in df.iterrows():
    model = update(model, obs['s'], obs['a'], obs['r'], obs['sp'])
policy = np.argmax(model.Q, axis=1) + 1
print(np.all(policy==1.0))
np.savetxt(f'data/{dataset}.policy', policy.astype(int), fmt='%i')

# epsilon greedy 15.9 to call update from simulate function

21960 49252
(50000, 7)
False
